References:
- https://platform.openai.com/docs/guides/speech-to-text/quickstart
- https://medium.com/nlplanet/transcribing-youtube-videos-with-whisper-in-a-few-lines-of-code-f57f27596a55

In [ ]:
#pip install openai-whisper
#pip install pytube
import pytube as pt
from pytube.exceptions import RegexMatchError
import whisper

In [ ]:
from tqdm.auto import tqdm

In [ ]:
import torch  # install steps: pytorch.org
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
yt = pt.YouTube("https://www.youtube.com/watch?v=aJjCjWdvmfo") #video length is 3:04 
stream = yt.streams.filter(only_audio=True)[0]
stream.download(filename="audio.mp3")

In [ ]:
model = whisper.load_model("medium").to(device)

In [ ]:
result = model.transcribe("audio.mp3")
print(result) 

In [ ]:
result['text']

In [ ]:
import translators as ts


In [ ]:
ts.translate_text(result['text'], 'google', to_language = 'en')

In [ ]:
result['segments']

In [ ]:
result['language']

In [ ]:
import os
os.remove("audio.mp3")

-------------------------------------------------------

In [ ]:
len(result['text'].split())

In [ ]:
result

----------------------------

In [1]:
import pandas as pd
import numpy as np
import csv
import os


In [2]:
import pytube as pt
import whisper
import translators as ts

In [ ]:
covid_vax_df = pd.read_csv("../datasets/covid_vaccine/videos.csv").drop("Unnamed: 0", axis=1)
covid_vax_df = covid_vax_df[['video_id','video_title','video_transcript']]
covid_vax_df.head()

In [ ]:
covid_ph_df = pd.read_csv("../datasets/covid_philippines/videos.csv").drop("Unnamed: 0", axis=1)
covid_ph_df = covid_ph_df[['video_id','video_title','video_transcript']]
covid_ph_df.head()

In [3]:
pal_is_df = pd.read_csv("../datasets/israel-palestine_conflict_history/videos.csv").drop("Unnamed: 0", axis=1)
pal_is_df = pal_is_df[['video_id','video_title','video_transcript']]
pal_is_df.head()

,video_id,video_title,video_transcript
0,R0ftmf_Uv9A,Brief history of Israel-Palestine conflict,NaN
1,Bno1m1zhIWs,How the Israeli-Palestinian Conflict Began | H...,NARRATOR: Israel. It's the only Jewish\r\nnat...
2,0anQIEeLonA,Israel-Palestine war: A simple history of how ...,at the break of dawn on the 7th of October 20...
3,iRYZjOuUnlU,"The Israel-Palestine conflict: a brief, simple...",One of the biggest myths about the Israel-Pal...
4,1wo2TLlMhiw,Conflict in Israel and Palestine through 2015:...,"Hi, I'm John Green, this is Crash Course Worl..."


In [4]:
import torch  # install steps: pytorch.org
device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("medium").to(device)


In [5]:
#so we can access age restricted videos
from pytube.innertube import _default_clients

_default_clients["ANDROID_MUSIC"] = _default_clients["ANDROID_CREATOR"]

In [144]:
def whisper_transcribe(input_df):
    
    df = input_df[['video_id','video_transcript']].copy()
    whis = np.array([])
    try:
        for row in range(len(input_df)):
            try:
                if pd.isna(input_df.at[row, 'video_transcript']):

                    video = 'https://www.youtube.com/watch?v=' + input_df['video_id'][row]
                    print("transcribing "+video)
                    yt = pt.YouTube(video,use_oauth=True) #use_oauth to access age restricted
                    stream = yt.streams.filter(only_audio=True)[0]
                    stream.download(filename="audio_" + input_df['video_id'][row]+".mp3")
                    audio_file = ("audio_"+ input_df['video_id'][row] +".mp3")
                    result = model.transcribe(audio_file)
                    os.remove("audio_" + input_df['video_id'][row]+".mp3")
                    if len(result['text'].split())>=10:
                        try:
                            en_res = ts.translate_text(result['text'], 'google', to_language = 'en')
                            whis =np.append(en_res,result['text'])
                        except:
                            whis =np.append(whis,result['text'])
                    else:
                        whis =np.append(whis,float(np.nan))
                else:
                    whis = np.append(whis,input_df['video_transcript'][row])

            except Exception as e:
                print("Something happened:", e)
                whis = np.append(whis,input_df['video_transcript'][row])
    except KeyboardInterrupt:
        print("interrupted")    
        #df['final_transcript'] = whis
        return whis

    df['final_transcript'] = whis
    return df

In [93]:
pal_is_df

,video_id,video_title,video_transcript
0,R0ftmf_Uv9A,Brief history of Israel-Palestine conflict,NaN
1,Bno1m1zhIWs,How the Israeli-Palestinian Conflict Began | H...,NARRATOR: Israel. It's the only Jewish\r\nnat...
2,0anQIEeLonA,Israel-Palestine war: A simple history of how ...,at the break of dawn on the 7th of October 20...
3,iRYZjOuUnlU,"The Israel-Palestine conflict: a brief, simple...",One of the biggest myths about the Israel-Pal...
4,1wo2TLlMhiw,Conflict in Israel and Palestine through 2015:...,"Hi, I'm John Green, this is Crash Course Worl..."
...,...,...,...
195,XYn5t0tuuAk,Arnold Schwarzenegger&#39;s &#39;Anti-Hate&#39...,hello everybody I want to talk to you today a...
196,Fo77sTGpngQ,Why Evangelical Christians Love Israel | VICE ...,the israeli-palestinian conflict has been the...
197,rzoI6RUG1iQ,Palestinians Protesting Jewish Settlements Cla...,NaN
198,JN4mnVLP0rU,Why the Middle East’s Borders Guarantee Foreve...,this video is made possible by nebula use the...


In [ ]:
whisper_trans_pal_is = whisper_transcribe(pal_is_df[:5])

In [ ]:
whisper_trans_pal_is

In [ ]:
final_pal_is =whisper_trans_pal_is.copy()

In [ ]:
whisper_trans_pal_is = whisper_transcribe(pal_is_df[15:].reset_index())

In [77]:
final_pal_is = pd.read_csv('../datasets/israel-palestine_conflict_history/partial_whisper_transcript.csv')

In [78]:
final_pal_is

,Unnamed: 0,video_id,video_transcript,final_transcript
0,0,R0ftmf_Uv9A,NaN,NaN
1,1,Bno1m1zhIWs,NARRATOR: Israel. It's the only Jewish\r\nnat...,NARRATOR: Israel. It's the only Jewish\r\nnat...
2,2,0anQIEeLonA,at the break of dawn on the 7th of October 20...,at the break of dawn on the 7th of October 20...
3,3,iRYZjOuUnlU,One of the biggest myths about the Israel-Pal...,One of the biggest myths about the Israel-Pal...
4,4,1wo2TLlMhiw,"Hi, I'm John Green, this is Crash Course Worl...","Hi, I'm John Green, this is Crash Course Worl..."
...,...,...,...,...
146,146,Y58njT2oXfE,what would happen if you build a refuge for A...,what would happen if you build a refuge for A...
147,147,WTjGtfVmIbA,Israel is at War it has come under terrorist ...,Israel is at War it has come under terrorist ...
148,148,q2209nFlQLo,look at this point you cannot turn on your ph...,look at this point you cannot turn on your ph...
149,149,qcKIjwiQgok,NaN,"When you think Palestine, you may think viole..."


In [52]:
final_pal_is = final_pal_is.drop("Unnamed: 0",axis =1)

In [145]:
whisper_trans_pal_is = whisper_transcribe(pal_is_df[183:].reset_index())

transcribing https://www.youtube.com/watch?v=1kYWII25cxM


d:\Program Files\miniconda3\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


transcribing https://www.youtube.com/watch?v=bU2wYxJSZBI
transcribing https://www.youtube.com/watch?v=rzoI6RUG1iQ


In [123]:
whis= whisper_trans_pal_is.copy() 
len(whis)

AttributeError: 'float' object has no attribute 'copy'

In [75]:
len(pal_is_df[151:182])

31

In [74]:
pal_is_df['video_transcript'][151]

nan

In [98]:
pal_is_df['video_id'][182]

'wVYZFNCmrBI'

In [129]:
whisp = pal_is_df[182:].drop('video_title',axis=1)
whisp['final_transcript'] =whis
whisp

In [130]:
whisp

,video_id,video_title,video_transcript,final_transcript
182,wVYZFNCmrBI,Complete story of Israel &amp; Palestine confl...,NaN,NaN


In [146]:
final_pal_is = pd.concat([final_pal_is,whisper_trans_pal_is],ignore_index=True)

In [147]:
final_pal_is

,video_id,video_transcript,final_transcript
0,R0ftmf_Uv9A,NaN,NaN
1,Bno1m1zhIWs,NARRATOR: Israel. It's the only Jewish\r\nnat...,NARRATOR: Israel. It's the only Jewish\r\nnat...
2,0anQIEeLonA,at the break of dawn on the 7th of October 20...,at the break of dawn on the 7th of October 20...
3,iRYZjOuUnlU,One of the biggest myths about the Israel-Pal...,One of the biggest myths about the Israel-Pal...
4,1wo2TLlMhiw,"Hi, I'm John Green, this is Crash Course Worl...","Hi, I'm John Green, this is Crash Course Worl..."
...,...,...,...
195,XYn5t0tuuAk,hello everybody I want to talk to you today a...,hello everybody I want to talk to you today a...
196,Fo77sTGpngQ,the israeli-palestinian conflict has been the...,the israeli-palestinian conflict has been the...
197,rzoI6RUG1iQ,NaN,nan
198,JN4mnVLP0rU,this video is made possible by nebula use the...,this video is made possible by nebula use the...


In [43]:
len(whisper_trans_pal_is)

20

In [44]:
len(pal_is_df[113:133])

20

In [132]:
final_pal_is=  final_pal_is.drop('Unnamed: 0',axis=1)

KeyError: "['Unnamed: 0'] not found in axis"

In [148]:
final_pal_is.to_csv('../datasets/israel-palestine_conflict_history/final_whisper_transcript.csv')

In [141]:
final_pal_is

,video_id,video_transcript,final_transcript
0,R0ftmf_Uv9A,NaN,NaN
1,Bno1m1zhIWs,NARRATOR: Israel. It's the only Jewish\r\nnat...,NARRATOR: Israel. It's the only Jewish\r\nnat...
2,0anQIEeLonA,at the break of dawn on the 7th of October 20...,at the break of dawn on the 7th of October 20...
3,iRYZjOuUnlU,One of the biggest myths about the Israel-Pal...,One of the biggest myths about the Israel-Pal...
4,1wo2TLlMhiw,"Hi, I'm John Green, this is Crash Course Worl...","Hi, I'm John Green, this is Crash Course Worl..."
...,...,...,...
178,7QNNClAYHFk,often the settlers and their flock try to cre...,often the settlers and their flock try to cre...
179,WM9rdc0fsgo,NaN,فلسطین اور اسرائیل کے کونفرکت کو تقریباً سو س...
180,gCNeDWCI0vo,NaN,nan
181,wznD7uCEcLk,Please welcome senior\r\nwriter at The Interc...,Please welcome senior\r\nwriter at The Interc...


In [50]:
final_cov_vax = pd.read_csv('../datasets/covid_vaccine/final_whisper_transcript.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/israel-palestine_conflict_history/final_whisper_transcript.csv'

In [ ]:
whisper_trans_covidvax = whisper_transcribe(covid_vax_df[100:].reset_index())

In [ ]:
whisper_trans_covidvax

In [ ]:
final_cov_vax = pd.concat([final_cov_vax,whisper_trans_covidvax],ignore_index=True)

In [ ]:
final_cov_vax.to_csv('../datasets/covid_vaccine/final_whisper_transcript.csv')

In [ ]:
whisper_trans_covidPH =whisper_transcribe(covid_ph_df[:5])

In [ ]:
whisper_trans_covidPH

In [ ]:
final_transcript = whisper_trans_covidPH.copy()

In [ ]:
covid_ph_df[5:50]

In [ ]:
whisper_trans_covidPH =whisper_transcribe(covid_ph_df[5:50].reset_index())

In [ ]:
whisper_trans_covidPH

In [ ]:
whisper_trans_covidPH['video_transcript'].isna().sum()

In [ ]:
whisper_trans_covidPH['final_transcript'].isna().sum()

In [ ]:
final_transcript = pd.concat([final_transcript,whisper_trans_covidPH],ignore_index=True)


In [ ]:
whisper_trans_covidPH =whisper_transcribe(covid_ph_df[50:100].reset_index())

In [ ]:
len(whisper_trans_covidPH)

In [ ]:
final_transcript = pd.concat([final_transcript,whisper_trans_covidPH],ignore_index=True)


In [ ]:
#final_transcript = final_transcript.reset_index(drop=True)
final_transcript.reset_index(drop=True,inplace=True)
final_transcript

In [ ]:

final_transcript.to_csv('../datasets/covid_philippines/whisper_transcript.csv') #saved it so no need to repeaat the whole thing if kernel crash

In [ ]:
whisper_trans_covidPH =whisper_transcribe(covid_ph_df[100:].reset_index())

In [ ]:
whisper_trans_covidPH

In [ ]:
final_transcript = pd.concat([final_transcript,whisper_trans_covidPH],ignore_index=True)


In [ ]:
final_transcript

In [ ]:
final_transcript.loc[final_transcript['final_transcript'] =='nan'] = float(np.nan)

In [ ]:
final_transcript['final_transcript'].isna().sum()

In [ ]:
covid_ph_df['video_transcript'].isna().sum()

In [ ]:
covid_ph_df

In [ ]:
final_transcript.to_csv('../datasets/covid_philippines/final_whisper_transcript.csv') #saved it so no need to repeaat the whole thing if kernel crash


In [ ]:
for i in range(len(covid_ph_df)):
    if covid_ph_df['video_id'][i]!=final_transcript['video_id'][i]:
        print(f"{i} covid ph {covid_ph_df['video_id'][i]} final transcript {final_transcript['video_id'][i]}")
        #break;

In [ ]:

print(final_transcript['video_id'][45:55])


In [ ]:
print(covid_ph_df['video_id'][45:55])

In [ ]:
covid_ph_df=covid_ph_df.drop(index=50,axis=0)
covid_ph_df.reset_index(drop=True,inplace=True)
covid_ph_df

In [ ]:
final_transcript = pd.read_csv("../datasets/covid_philippines/final_whisper_transcript.csv").drop("Unnamed: 0", axis=1)

In [ ]:
covid_ph_df

In [ ]:
len(final_transcript)

In [ ]:
len(covid_ph_df)

In [ ]:
covid_ph_df.loc[final_transcript['video_id'].isna()]

In [ ]:
#whisper_trans_covidPH =whisper_transcribe(covid_ph_df.loc[final_transcript['video_id'].isna()].reset_index())

In [ ]:
#final_transcript['video_id'].isna().sum()

In [ ]:
final_transcript[final_transcript['video_id'].isna()]

In [ ]:
whisper_trans_covidPH

In [ ]:
#somee video_id was nan at some point disregard this
'''import math
w= whisper_trans_covidPH.copy()
final =final_transcript.copy()

j=0
for i in range(len(final)):
    if final['video_id'][i]==None or final['video_id'][i]==np.nan:
        final.loc[i,'video_id'] = whisper_trans_covidPH['video_id'][j]
        final.loc[i,'video_transcript'] = whisper_trans_covidPH['video_transcript'][j]
        final.loc[i,'final_transcript'] = whisper_trans_covidPH['final_transcript'][j]
        j+=1
    elif type(final['video_id'][i]) ==float and math.isnan(final['video_id'][i]):
        final.loc[i,'video_id'] = whisper_trans_covidPH['video_id'][j]
        final.loc[i,'video_transcript'] = whisper_trans_covidPH['video_transcript'][j]
        final.loc[i,'final_transcript'] = whisper_trans_covidPH['final_transcript'][j]
        j+=1
    #j+=1'''

In [ ]:
final_transcript.to_csv('../datasets/covid_philippines/final_whisper_transcript.csv') #saved it so no need to repeaat the whole thing if kernel crash
